Import JSON files and merge them to one pandaframe 


In [1]:
import numpy as np
import json
import pandas as pd
import os
import datetime

In [6]:
path = 'C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/'
path_gtfs ='C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3sept.zip'

In [7]:
# creating a list conatining all my json files
list_files=[]
for i in os.listdir(path):
    if i.endswith('.json'):
        list_files.append(path + i)

#print(list_files)

In [8]:
#merge the data into one pandaframe which contains every record of timestamp, line id, direction id, point id and the distance from the last point 
listvehiculepositions=[]
for file in list_files:#je parcours ma liste cntenant mes json files
    #print('Uploading the file' + ':' +file )
    with open(file) as f : 
        data = json.load(f) #je load les json files
        f.close()
        for d in data['data']:    
            for line in d['Responses']:  
                if line is not None : # la 10ième fois qu'on appelle l'API il y a juste le temps et rien dans l'élément lines donc on vérifie que ce ne soit pas pas vide
                    for position in line['lines']:  
                        for i in position['vehiclePositions']:   
                            d_time = d['time']
                            position_lineId = position['lineId']
                            i_directionId = i['directionId']
                            i_distancefromPoint = i['distanceFromPoint']
                            i_pointId = i['pointId']
                            listvehiculepositions.append([d_time,position_lineId,i_directionId,i_distancefromPoint,i_pointId]) #construction de la liste des listes
print('Uploading of the files is finished')
df_vehiculepositions = pd.DataFrame(data=listvehiculepositions, columns = ['time','lineId', 'directionId','distanceFromPoint','pointId'])# on va en faire un panda frame 

Uploading of the files is finished


In [ ]:
print(df_vehiculepositions.lineId.value_counts())

In [9]:
df_vehiculepositions.time = pd.to_numeric(df_vehiculepositions.time, downcast = "float")

##Division par 1000 pour passer de ms à s /!\ Attention à le faire qu'une fois
df_vehiculepositions.time = df_vehiculepositions.time/1000 

In [10]:
import datetime
#df_vehiculepositions['time'].sort_values()
def datefromtimestamp(timestamp):
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')
    return date
time_fun = datefromtimestamp(df_vehiculepositions.time[0])
print(time_fun)
my_date = datetime.datetime(2021,9,3)
print(my_date)
my_stamp = datetime.datetime.timestamp(my_date)
print(my_stamp)


09-06-2021 09:54:46
2021-09-03 00:00:00
1630620000.0


Import gtfs line

In [11]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import matplotlib.pyplot as plt

In [12]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(path_gtfs, busiest_date=False)

In [ ]:
stop_times.head()

In [ ]:
df_vehiculepositions.head()

In [13]:
array = df_vehiculepositions.index.to_numpy()
date_init = datetime.datetime(2021, 9, 6, 4)
epoch_date_init = datetime.datetime.timestamp(date_init)
date_end = datetime.datetime(2021, 9, 7, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
epoch_date_end = datetime.datetime.timestamp(date_end)
print(epoch_date_init)
print(df_vehiculepositions.time[0])

    
    

1630893600.0
1630914886.924


In [14]:
df_dict = df_vehiculepositions.to_dict()

In [16]:
import datetime
from tqdm import tqdm
import math
import matplotlib.pylab as plt

def actual_time(line, stop, year, month, day):
    
    date_init = datetime.datetime(year, month, day, 4)
    epoch_date_init = datetime.datetime.timestamp(date_init)
    date_end = datetime.datetime(year, month, day+1, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
    epoch_date_end = datetime.datetime.timestamp(date_end)
    date_midnight = datetime.datetime(year, month, day, 0) #Obtain seconds after midnight format
    midnight_time = datetime.datetime.timestamp(date_midnight)
    listvehiculearrivals = []
    humanlistvehiculearrivals = []
    d_distrib = []
    for index in tqdm(df_dict['time']):
        if epoch_date_init < df_dict['time'][index]:
            if epoch_date_end > df_dict['time'][index]:
                if df_dict['lineId'][index] == line:
                     if df_dict['pointId'][index] == stop:
                        if math.isclose(df_dict['distanceFromPoint'][index], 0,abs_tol = 40):
                            d_distrib.append(df_dict['distanceFromPoint'][index])
                            sec_after_midnight = df_dict['time'][index] - midnight_time
                            human_arrival_time = datefromtimestamp(df_dict['time'][index])
                            listvehiculearrivals.append(sec_after_midnight)
                            humanlistvehiculearrivals.append(human_arrival_time)
    return listvehiculearrivals, humanlistvehiculearrivals, d_distrib

machine_timetable, timetable, d_distrib = actual_time("7", "5312", 2021, 9, 10)
len(timetable)
    

100%|██████████| 19421883/19421883 [00:02<00:00, 7972439.00it/s]


134

In [10]:
calendar_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar.txt', sep =',')
calendar_dates_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar_dates.txt', sep = ',')

calendar_df.set_index('service_id', inplace = True)
calendar_dates_df1 = calendar_dates_df[calendar_dates_df['exception_type'] == 1]
calendar_dates_df1.head()

,service_id,date,exception_type
0,237476501,20210912,1
1,237566502,20210919,1
2,238162502,20210919,1
3,238164502,20210919,1
4,238234501,20210919,1


In [90]:
import datetime
def date_to_day_of_week(date_str):
  # Parse the date string into a datetime object
  date = datetime.datetime.strptime(date_str, '%Y%m%d')

  # Return the day of the week (Monday, Tuesday, etc.)
  return date.strftime('%A').lower()

dw = date_to_day_of_week('20210912')
calendar_dates_df['service_id']

0      237476501
1      237566502
2      238162502
3      238164502
4      238234501
         ...    
306    235954071
307    235005071
308    237243501
309    235536070
310    238255501
Name: service_id, Length: 311, dtype: int64

In [111]:
import pandas as pd
import datetime

calendar_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar.txt', sep =',')
calendar_dates_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar_dates.txt', sep = ',')

def update_calendar(calendar_df, calendar_dates_df):
    calendar_dates_df1 = calendar_dates_df[calendar_dates_df['exception_type'] == 1]
    calendar_dates_df2 = calendar_dates_df[calendar_dates_df['exception_type'] == 2]
    
    calendar_df.set_index('service_id', inplace = True)

    for index, elem in calendar_dates_df1.iterrows():
        date1 = str(calendar_dates_df1['date'][index])
        weekday = date_to_day_of_week(date1)
        service2change = calendar_dates_df1['service_id'][index]
        calendar_df.loc[service2change, weekday] = 1
        
    for index, elem in calendar_dates_df2.iterrows():
        date = str(calendar_dates_df2['date'][index])
        weekday = date_to_day_of_week(date)
        service2change = calendar_dates_df2['service_id'][index]
        calendar_df.loc[service2change, weekday] = 0

    return calendar_df

calendar_df = update_calendar(calendar_df, calendar_dates_df)
calendar_df[5:10]


,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
service_id,,,,,,,,,
237269601,0,0,0,0,0,0,1,20210905,20210905
237476501,0,0,0,0,0,1,1,20210912,20210912
237566502,0,0,0,0,0,1,1,20210919,20210919
236892052,1,1,1,1,1,0,0,20210823,20210827
234566500,0,0,0,0,0,1,0,20210828,20210828


In [95]:
for index, elem in calendar_dates_df1.iterrows():
        date1 = str(calendar_dates_df1['date'][index])
        weekday = date_to_day_of_week(date1)
        service2change = calendar_dates_df1['service_id'][index]
        calendar_df.loc[service2change, weekday] = 1
calendar_df[5:10]

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
service_id,,,,,,,,,
237269601,0,0,0,0,0,0,1,20210905,20210905
237476501,0,0,0,0,0,1,1,20210912,20210912
237566502,0,0,0,0,0,1,1,20210919,20210919
236892052,1,1,1,1,1,0,0,20210823,20210827
234566500,0,0,0,0,0,1,0,20210828,20210828


In [ ]:
import pandas as pd
import datetime

calendar_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar.txt', sep =',')
calendar_dates_df = pd.read_table('C:/Users/mehdi/Desktop/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar_dates.txt', sep = ',')
def date_to_day_of_week(date_str):
  # Parse the date string into a datetime object
  date = datetime.datetime.strptime(date_str, '%Y%m%d')

  # Return the day of the week (0=Monday, 1=Tuesday, etc.)
  return date.strftime()

def update_calendar(calendar_df, calendar_dates_df):
    calendar_dates_df1 = calendar_dates_df[calendar_dates_df['exception_type'] == 1]
    calendar_dates_df2 = calendar_dates_df[calendar_dates_df['exception_type'] ==2]
    calendar_df.set_index('service_id', inplace = True)

    for elem in calendar_dates_df1:
        date = calendar_dates_df1['date'][elem]
        weekday = date_to_day_of_week(date)
        service2change = calendar_dates_df1['service_id'][elem]
        calendar_df.loc[service2change, weekday] == '1'
        
    for elem in calendar_dates_df2:
        date = calendar_dates_df2['date'][elem]
        weekday = date_to_day_of_week(date)
        service2change = calendar_dates_df2['service_id'][elem]
        calendar_df.loc[service2change, weekday] == '0'



def CheckRange(Start,End,date):
    return int(date) in range(int(Start),int(End))

def FromDayToService(date):
    #date est un tuple ('20210907','tuesday')
    ListServiceId=[]

    for i in range(len(calendardf)):

        startdate,endate=calendardf.iloc[i,[8,9]]
        
        if CheckRange(startdate,endate+1,date[0]):

            if calendardf.loc[i,date[1]]==1:
                
                ListServiceId.append(str(calendardf.iloc[i,0]))

    return ListServiceId

def FromLineIdtoRouteId(LineId):
    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def SelectInGTFS(day,lineId,stopid):
    ListServiceId = FromDayToService(day)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopid)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListServiceId))].sort_values('arrival_time')
    Comparaisondf=Comparaisondf[['trip_id','arrival_time','stop_id','route_id','service_id']]
    ArrivalTime = Comparaisondf['arrival_time'].to_numpy()
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Comparaisondf['Interval']= IntervalTime
    Comparaisondf['Ponctuality']=np.array(IntervalTime)>720
    return Comparaisondf

def ScheduledTime(lineId,stopId,date):
    ListServiceId = FromDayToService(date)
    RouteId = FromLineIdtoRouteId(lineId)
    Comparaisondf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListServiceId))].sort_values('arrival_time')
    ArrivalTime = Comparaisondf['arrival_time'].to_numpy()
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime.insert(0,ArrivalTime[0])
    Punctuality = np.array(IntervalTime)>720 
    IntervalTime = list(zip(IntervalTime,Punctuality))

    return IntervalTime



In [ ]:
calendar_dates = pd.read_table('C:/Users/midou/OneDrive/Bureau/Big Data/Data mining/Project/Project Data-20221115/gtfs3Sept/calendar_dates.txt', sep = ',')
calendar_dates.date = pd.to_datetime(calendar_dates.date.astype(str),format = '%Y%m%d')
calendardf.start_date = pd.to_datetime(calendardf.start_date.astype(str), format = "%Y%m%d")
calendardf.end_date = pd.to_datetime(calendardf.end_date.astype(str), format = "%Y%m%d")




In [ ]:
calendar_df.head()
calendard_f.loc[234578052]

# Import the necessary libraries
import datetime

# Define a function to convert a date in YYYYMMDD format to the day of the week
def date_to_day_of_week(date_str):
  # Parse the date string into a datetime object
  date = datetime.datetime.strptime(date_str, '%Y%m%d')

  # Return the day of the week (0=Monday, 1=Tuesday, etc.)
  return date.weekday()

# Test the function with some example dates
print(date_to_day_of_week('20221210')) # 6 (Saturday)
print(date_to_day_of_week('20221211')) # 0 (Sunday)
print(date_to_day_of_week('20221212')) # 1 (Monday)

# Convert the date to a datetime object
date = pd.to_datetime('20221210')

# Check if the service is available on the given date
service_id = 12345
service_available = False

# Check the calendar for the given service ID
calendar_service = calendar_df[calendar_df['service_id'] == service_id]

# Check if the service is available on the given day of the week
day_of_week = date.weekday()
if calendar_service[f'd{day_of_week+1}'].bool():
  service_available = True
  calendardf[service_id]

# Check the calendar_dates for exceptions to the regular calendar
calendar_date_exceptions = calendar_dates_df[calendar_dates_df['service_id'] == service_id]

# Check if there are any exceptions on the given date
if calendar_date_exceptions[calendar_date_exceptions['date'] == date].size > 0:
  # If there is an exception on the given date, check if the service is added or removed
  exception_type = calendar_date_exceptions[calendar_date_exceptions['date'] == date]['exception_type'].iloc[0]
  if exception_type == 1:
    # If the service is added on the given date, the service is available
    service_available = True
  elif exception_type == 2:
    # If the service is removed on the given date, the service is not available
    service_available = False

# Print the result
if service_available:
  print('Service is available on the given date.')
else:
  print('Service is not available on the given date.')
        


In [ ]:
Date=('20210918','saturday')
lineid = '4'
stopid = '5026'


test = ScheduledTime(lineid,stopid,Date)
test